# Othello Computer Vision Project

In [2]:
%pip install numpy matplotlib scipy opencv-python opencv-python-headless Pillow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Step 1: Import Libraries

In [3]:
# Import all libraries here

import sys
if sys.version_info[0] < 3:
	raise Exception("Python 3 not detected.")
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from scipy import io
import scipy
import cv2


Create a function to:
1. Load the image
2. Get the corners of the image
3. Divide it into the 4x4 grid
4. Analyze the pixels for each grid square
5. Determine if the piece is black or white or empty
6. Create the game state string
7. Return the state of the game

## Step 2: Load Image & Edge Detection

In [4]:
# Load the image
image = cv2.imread('images/random-board-gamesman-uni.png')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise and improve edge detection
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Detect edges using Canny edge detector
edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

# Find contours based on edges detected
contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Assuming the largest contour is the board, find its bounding box
board_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(board_contour)

# Crop the image to the board's bounding box (optional, for further processing)
board = image[y:y+h, x:x+w]

cv2.imshow('Board', board)
cv2.waitKey(0)
cv2.destroyAllWindows()

2024-03-06 12:07:28.684 Python[23286:5179037] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


## Step 3: Grid Detection and Piece Recognition

In [5]:
# w is the width from above
# h is the height of the image from above

grid_size = 4
square_width, square_height = w // grid_size, h // grid_size

# 0 for empty, 1 for black, 2 for white
game_state = np.zeros((grid_size, grid_size), dtype=int)  

# Convert the whole board image to HSV for better color segmentation
hsv_board = cv2.cvtColor(board, cv2.COLOR_BGR2HSV)

# Define color ranges for black and white pieces
# Note: These ranges might need adjustment based on your lighting conditions and camera
black_lower, black_upper = np.array([0, 0, 0]), np.array([180, 255, 50])
white_lower, white_upper = np.array([0, 0, 200]), np.array([180, 25, 255])

# Create masks for black and white for the whole board
black_mask = cv2.inRange(hsv_board, black_lower, black_upper)
white_mask = cv2.inRange(hsv_board, white_lower, white_upper)


# What we hope to accomplish
# Convert the whole board image to a mask to determine black pieces
# Convert the whole board image to a mask to determine white pieces 
for i in range(grid_size):  # i for rows
    for j in range(grid_size):  # j for columns
        # Define the pixel range for the current square
        start_x, start_y = j * square_width, i * square_height
        end_x, end_y = (j + 1) * square_width, (i + 1) * square_height

        # Access the relevant section of the black and white masks
        black_mask_square = black_mask[start_y:end_y, start_x:end_x]
        white_mask_square = white_mask[start_y:end_y, start_x:end_x]

        # Determine if square is black, white, or empty based on the mask
        if np.sum(black_mask_square) > 100:  # Adjust threshold based on your conditions
            game_state[i, j] = 'b'  # Black piece, noting [i, j] for row i, column j
        elif np.sum(white_mask_square) > 100:  # Adjust threshold based on your conditions
            game_state[i, j] = 'w'  # White piece, noting [i, j] for row i, column j
        else:
            game_state[i, j] = '-'

ValueError: invalid literal for int() with base 10: '-'

## Step 4: Convert Gamestate 2D Array into 1D String

In [ ]:
# Flatten the 2D array into 1D array
flattened_game_state = game_state.flatten(game_state.shape[0])

# Use the .join method to convert to a string
game_state_string = ''.join(flattened_game_state)

print(game_state_string)

# Combined Pipeline Function

In [ ]:
def get_gamestate_from_board_image(url):
  # -----------------------------------------------
  # Step 1: Load Image
  
  # Load the image
  image = cv2.imread(url)

  # -----------------------------------------------
  # Step 2: Edge Detection

  # Convert to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Apply GaussianBlur to reduce noise and improve edge detection
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)

  # Detect edges using Canny edge detector
  edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

  # Find contours based on edges detected
  contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Assuming the largest contour is the board, find its bounding box
  board_contour = max(contours, key=cv2.contourArea)
  x, y, w, h = cv2.boundingRect(board_contour)

  # Crop the image to the board's bounding box (optional, for further processing)
  board = image[y:y+h, x:x+w]

  cv2.imshow('Board', board)
  cv2.waitKey(0)
  cv2.destroyAllWindows()


  # -----------------------------------------------
  # Step 3: Grid Detection and Piece Recognition
  
  # w is the width from above
  # h is the height of the image from above

  grid_size = 4
  square_width, square_height = w // grid_size, h // grid_size

  # 0 for empty, 1 for black, 2 for white
  game_state = np.zeros((grid_size, grid_size), dtype=int)  

  # Convert the whole board image to HSV for better color segmentation
  hsv_board = cv2.cvtColor(board, cv2.COLOR_BGR2HSV)

  # Define color ranges for black and white pieces
  # Note: These ranges might need adjustment based on your lighting conditions and camera
  black_lower, black_upper = np.array([0, 0, 0]), np.array([180, 255, 50])
  white_lower, white_upper = np.array([0, 0, 200]), np.array([180, 25, 255])

  # Create masks for black and white for the whole board
  black_mask = cv2.inRange(hsv_board, black_lower, black_upper)
  white_mask = cv2.inRange(hsv_board, white_lower, white_upper)


  # What we hope to accomplish
  # Convert the whole board image to a mask to determine black pieces
  # Convert the whole board image to a mask to determine white pieces 
  for i in range(grid_size):  # i for rows
      for j in range(grid_size):  # j for columns
          # Define the pixel range for the current square
          start_x, start_y = j * square_width, i * square_height
          end_x, end_y = (j + 1) * square_width, (i + 1) * square_height

          # Access the relevant section of the black and white masks
          black_mask_square = black_mask[start_y:end_y, start_x:end_x]
          white_mask_square = white_mask[start_y:end_y, start_x:end_x]

          # Determine if square is black, white, or empty based on the mask
          if np.sum(black_mask_square) > 100:  # Adjust threshold based on your conditions
              game_state[i, j] = 'b'  # Black piece, noting [i, j] for row i, column j
          elif np.sum(white_mask_square) > 100:  # Adjust threshold based on your conditions
              game_state[i, j] = 'w'  # White piece, noting [i, j] for row i, column j
          else:
              game_state[i, j] = '-'

  # Flatten the 2D array into 1D array
  flattened_game_state = game_state.flatten(game_state.shape[0])

  # Use the .join method to convert to a string
  game_state_string = ''.join(flattened_game_state)

  return game_state_string

# Create List of AutoGUI Moves from Recording

Look at the first frame of your original game state position string
Save the current game position string
The user makes their move
Once the pieces have stopped updating, then after x milliseconds for the animated version to change, then look at frame after (I estimate between 30 to 35 milliseconds for the update to occur)
Save the post-move game position string
Look at the difference between the two game state strings, where the grid square has changed from a ‘-’, empty spot, to either ‘W’ or ‘B’ depending on who’s turn it was for example:
--B--BB--WWW---- 
--B--BB--WBW---B
We conclude that the player added a new piece in index 15
Construct the autoguiDovie
Therefore, the game move was:
autoguiMove: A_h_15_x
Repeat this process through analyzing the entire game’s recording
Build a list of the autogui Moves 

1. Get a before and after frame of the image
2. Calculate game strings
3. Calculate difference between game strings

In [ ]:
def get_move_from_two_frames(url1, url2):
  # Load the two frames
  # frame1 = cv2.imread(url1)
  # frame2 = cv2.imread(url2)

  str1 = get_gamestate_from_board_image(url1)
  str2 = get_gamestate_from_board_image(url2)

  # Compare both the strings for a difference
  # Ensure both strings are of the same length
  if len(str1) != len(str2):
      print("Error: Strings are of different lengths.")
      return

  move_index = None
  check_count = 0

  # Iterate through each character in the strings
  for i in range(len(str1)):
      if str1[i] == '-' and str2[i] in ['B', 'W']:
          move_index = i
          print(f"Position {i}: '-' changed to '{str2[i]}'")
          check_count += 1
    
  # In case we made an error
  if (check_count > 1):
      print("We made 2 or more changes to determining the move here")
      print(str(check_count))
  
  # Create the auto_gui move
  auto_gui_move = 'A_h_' + str(move_index) + '_x'

  return auto_gui_move

## Iterate through video & identify frames for update

In [ ]:
def extract_and_save_all_frames(video_path, output_folder):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    while True:
        # Read the next frame
        ret, frame = cap.read()
        
        # If frame is read correctly ret is True
        if not ret:
            break  # Exit the loop if there are no frames left to read
        
        # Construct the output filename
        output_filename = f"{output_folder}/frame_{frame_count:04d}.jpg"
        
        # Save the frame to disk
        cv2.imwrite(output_filename, frame)
        
        # Increment the frame count
        frame_count += 1
    
    # Release the video capture object
    cap.release()

    print(f"Total frames saved: {frame_count}")

# Example usage
video_path = 'path_to_your_video.mp4'
output_folder = 'path_to_output_folder'
extract_and_save_all_frames(video_path, output_folder)


In [ ]:
game_states = []
auto_gui_moves = []

# Pass in the auto gui moves